##Задача всего проекта
Сегодня у клиентов медицинских учреждений существует запрос на круглосуточный доступ к медицинским сервисам и данным о своём здоровье. Подобные сервисы могут помогать пациентам интерпретировать результаты анализов, проводить первичную диагностику по описываемым симптомам.

В основе таких предсказательных сервисов лежат алгоритмы искусственного интеллекта, которые обучаются на основе реальных данных. Сбор подобных данных — ключевой этап разработки алгоритма искусственного интеллекта.

Часто медицинские данные представляют из себя текстовые протоколы пациентов, которые необходимо дополнительно обработать перед использованием. Этот процесс включает извлечение полезных характеристик: например, конкретных симптомов.

Разработать прототип медицинского рекомендательного алгоритма на основе машинного обучения.



##Задача 2 этапа:
Создать новые признаки на основе имеющихся данных, которые впоследствии будут использоваться для обучения нашей модели для столбца "ПеренесенныеЗаболевания".

##Признаки:

1) MedicalRecordKey - int64, идентификатор медицинской записи, например 106627476

2) MedicalRecordDate - object, дата и время медицинской записи, например 2023-09-01 18:39:46.000

3) PatientKey - object, уникальный номер пациента, например 0xD8800629A4ADB5D792711AC5F17F72859E0F05FEB21E...

4) Возраст - int64,в днях, например 9764

5) Пол - float64,в бинарном значении, например 1.0

6) СпециальностьВрача - object, например Врач уролог

7) Жалобы - object, например На покраснение и шелушение кожи головы.

8) ПеренесенныеЗаболевания - object, например ГЭРБ ЖДА

9) ПеренесенныеОперации - object, например Стентирование коронарных артерий 05.02.2019г.

10) ПринимаемыеПрепараты - object, например монурал, канефрон.

##Библиотеки:

In [1]:
pip install pymorphy2

In [2]:
pip install nltk

In [3]:
pip install natasha

In [4]:
import pandas as pd
import numpy as np
import pymorphy2
import re
from collections import Counter
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import os
import natasha
from pymystem3 import Mystem

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
from natasha import (
    Segmenter,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,

    Doc
)

segmenter = Segmenter()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)

In [6]:
morph = pymorphy2.MorphAnalyzer()

##Конец библиотек

In [7]:
#Опции для датасета
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [8]:
#Подключение гугл диска
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
#Создание датафрейма
try:
    df = pd.read_csv("/content/drive/MyDrive/Hakaton/ж/медицинские_протоколы.csv", sep=';')
except FileNotFoundError:
    try:
        df = pd.read_csv("/content/drive/MyDrive/syrius_AI_medicine/reports.csv", sep=';')
    except FileNotFoundError:
        df = pd.read_csv("/content/drive/MyDrive/data кванториум/reports.csv", sep = ';')

In [10]:
#Первичный просмотр
def cool_output(s, n):
    line = "*" * n
    padding = "-" * ((n - len(s)) // 2)
    if (n - len(s)) % 2 == 1:
        padding += "-"
    output = f"{line}\n{padding}{s}{padding}\n{line}"
    print(output)

def find_explicit_duplicates(df):
    explicit_duplicates = df[df.duplicated()]
    if not explicit_duplicates.empty:
        cool_output("Явные дубликаты", 80)
        display(explicit_duplicates)
    else:
        cool_output("Явных дубликатов не найдено", 80)

def allInfo(df):
    cool_output("Информация", 80)

    df_info = df.info()
    cool_output("Типы данных", 80)
    display(df.dtypes.value_counts())

    cool_output("Размеры", 80)
    print(df.shape)

    cool_output("Первые три строки", 10 * len(df.columns))
    display(df.head(3))

    cool_output("Последние три строки", 10 * len(df.columns))
    display(df.tail(3))

    cool_output("Случайные три строки", 10 * len(df.columns))
    display(df.sample(3))

    cool_output("Уникальные значения", 80)
    for col in df.columns:
        unique_count = len(df[col].unique())
        unique_percentage = round(unique_count * 100 / df.shape[0], 2)
        print(f'"{col}": уникальных значений: {unique_count} ({unique_percentage}%)')

    cool_output("Пропуски", 80)
    for col in df.columns:
        missing_count = len(df[df[col].isna()])
        missing_percentage = round(missing_count * 100 / df.shape[0], 2)
        print(f'"{col}": пропусков: {missing_count} ({missing_percentage}%)')

    find_explicit_duplicates(df)

    cool_output("Статистика", 10 * len(df.columns))
    display(df.describe(include="all"))

    footer = "".center(10 * len(df.columns), '-')
    cool_output("", 10 * len(df.columns))

#allInfo(df)

##Первичный просмотр показал:


1) Пропуски были найдены в столбцах: Пол, "ПеренесенныеЗаболевания", ПеренесенныеОперации, ПринемаемыеПрепараты.

2) Названия столбцов MedicalRecordKey, MedicalRecordDate, PatientKey на английском языке, а Возраст, Пол, СпециальностьВрача, Жалобы, "ПеренесенныеЗаболевания", ПеренесенныеОперации, ПринимаемыеПрепараты на русском языке, нужно перевести названия всех столбцов в нижний регистр и на английский язык, удалить лишние знаки препинания.
##Задания для 2 этапа:
1) Выбрать 1 из двух текстовых колонок - "Жалобы" или "ПеренесенныеЗаболевания".

2) Провести простейший парсинг данных: разбить текст по знакам препинания (лучше - только  по запятым), привести текст к нижнему регистру и обрезать пробелы в начале и конце.

3) Получить на выходе список из Жалоб или Заболеваний (могут быть как из одного, так и из нескольких слов), отсортированный по частотности, встречаемости в тексте.

4) Отобрать топ-10 самых частых Жалоб или Заболеваний, учитывая суммарную частотность их различного написания (с сокращениями и опечатками).

5) Создать для полученного топ-10 правила продвинутого парсинга, объединяющего различные варианты написания Жалоб или Заболеваний и учитывающего морфологию слов. Можно использовать такие библиотеки для работы с русским языком как natasha.

6) С помощью построенных правил распарсить Жалобы или Заболевания и преобразовать их в новые бинарные признаки (новые колонки), где 1 будет означать наличие, а 0 - отсутствие соответствующих Жалоб или Заболеваний.

In [11]:
#Переименование столбцов по стандарту PEP-8
df=df.rename(columns=dict((i, i.lower()) for i in list(df)))

df=df.rename(columns={
    "medicalrecordkey":"record_id",
    "medicalrecorddate":"date",
    "patientkey":"patient_id",
    "возраст":'age',
    "пол":"sex",
    "специальностьврача":"speciality_of_doctor",
    "жалобы":"problems",
    "перенесенныезаболевания":"past_illnesses",
    "перенесенныеоперации":"postponed_operations",
    "принимаемыепрепараты":"medicicals_taken"
})

In [12]:
#Удаление лишних столбцов
df=df.drop('patient_id', axis=1)
df=df.drop('record_id', axis=1)
df=df.drop('date', axis=1)

In [13]:
#Перевод возраста из дней в годы
def divise(row):
  return row["age"] // 365
df['age'] = df.apply(divise, axis=1)

In [14]:
#Проверка
df.head(100)

,age,sex,speciality_of_doctor,problems,past_illnesses,postponed_operations,medicicals_taken
0,26,1.0,Врач акушер гинеколог,на мажущие выделения из половых путей,"Факт получения травм, ожогов, ран, проведения медицинских манипуляций (инъекций, стоматологических, хирургических вмешательств и др.) в течение последних 30 дней отрицает, установление имплантата в течении 1 года отрицает. Перенесенные инфекционные заболевания, в т.ч. ВИЧ, вирусные гепатиты, туберкулез, COVID-19 отрицает.",Отрицает,Левотироксин 75 мкг/сут
1,37,1.0,Врач уролог,на дискомфорт при мочеиспускании на учащенное мочеиспускание,"Факт получения травм, ожогов, ран, проведения медицинских манипуляций (инъекций, стоматологических, хирургических вмешательств и др.) в течение последних 30 дней отрицает, установление имплантата в течении 1 года отрицает. Перенесенные инфекционные заболевания, в т.ч. ВИЧ, вирусные гепатиты, туберкулез, COVID-19 отрицает.",аппендэктомия 2013. кесарево сечение - преэклампсия (2018).,"монурал, канефрон."
2,28,0.0,Врач дерматовенеролог,На покраснение и шелушение кожи головы.,ГЭРБ ЖДА,-,Колекальльциферол - 4000 МЕ Калия йодид Мальтофер - 2 т/день
3,62,1.0,Врач кардиолог,"Подписано информированное добровольное согласие на получение первичной медико-санитарной помощи. Прием кардиолога повторно ЖАЛОБЫ на фоне стресса повышение АД 145/90 ммрттс с головными болями 2 балла по ВАШ, на эпизоды учащенного сердцебиения без четкой связи с физической нагрузкой, перебои в работе сердца -",узловой зоб,"аппендицит в 15-летнем возрасте, удаление кисты бартолиниевой железы в 2007 г., в 2022г - мастэктоия слева","на пенициллиновый ряд - отек квинке, сульфаниламиды- крапивница. На метронидазол - чувство дурноты"
4,42,1.0,Врач акушер гинеколог,Жалобы на болезненность и нагрубание правой молочной железы.,"Факт получения травм, ожогов, ран, проведения медицинских манипуляций (инъекций, стоматологических, хирургических вмешательств и др.) в течение последних 30 дней отрицает, установление имплантата в течении 1 года отрицает. Перенесенные инфекционные заболевания, в т.ч. ВИЧ, вирусные гепатиты, туберкулез, COVID-19 отрицает.",аппендэктомия в 5 лет Импланты молочных желез - 2016 лазерная коррекция миопии 2017,Не отягощен
5,35,1.0,Врач травматолог ортопед,на боли в левом коленном суставе.,"Факт получения травм, ожогов, ран, проведения медицинских манипуляций (инъекций, стоматологических, хирургических вмешательств и др.) в течение последних 30 дней отрицает, установление имплантата в течении 1 года отрицает. Перенесенные инфекционные заболевания, в т.ч. ВИЧ, вирусные гепатиты, туберкулез, COVID-19 отрицает.",лазерная коррекция зрения,не отягощен
6,36,0.0,Врач кардиолог,"на повышение АД до 150\100 мм рт ст,","Факт получения травм, ожогов, ран, проведения медицинских манипуляций (инъекций, стоматологических, хирургических вмешательств и др.) в течение последних 30 дней отрицает, установление имплантата в течении 1 года отрицает. Перенесенные инфекционные заболевания, в т.ч. ВИЧ, вирусные гепатиты, туберкулез, COVID-19 отрицает.",отрицает,NaN
7,58,1.0,Врач гастроэнтеролог,"Жалобы на отрыжку, периодическую тяжесть в животе-эпигастральной области, правом подреберье, после еды, периодическое вздутие живота , дискомфорт при дефекации","Факт получения травм, ожогов, ран, проведения медицинских манипуляций (инъекций, стоматологических, хирургических вмешательств и др.) в течение последних 30 дней отрицает, установление имплантата в течении 1 года отрицает. Перенесенные инфекционные заболевания, в т.ч. ВИЧ, вирусные гепатиты, туберкулез, COVID-19 отрицает.","мастэктомия левой м/ж июль 2022, комбинированное лечение",NaN
8,40,1.0,Врач невролог,"Жалобы на частые головные боли, ноющие, по ВАШ на 5-6б, чаще в затылочной области, до тошноты, двоение в глазах, появление молний, лучей и вспышек, чаще утром, фото и фонофобия присутствует, при движении головная боль усиливается. Жалобы на боли в шейном, поясничном отделе позвоночника по ВАШ на 5-6. иногда онемение левой руки, периоди

In [15]:
#Проверка размеров
df.shape

(12765, 7)

In [16]:
#Подключение набора данных с докторами
try:
    df_doctor_kinds = pd.read_csv("/content/drive/MyDrive/syrius_AI_medicine/doctor_types.csv", sep=';')
except FileNotFoundError:
    df_doctor_kinds = pd.read_csv("/content/drive/MyDrive/data кванториум/Копия doctor_types.csv", sep=';')

df_doctor_kinds = df_doctor_kinds.drop('PID', axis=1)
df_doctor_kinds = df_doctor_kinds.drop('SERVICE_FIELD', axis=1)

In [17]:
#Создание целочисленного признака для специальности врача
porydkoviy=100000
def doctor_id(row):
  global porydkoviy
  id=-1
  try:
    id=int(df_doctor_kinds.loc[df_doctor_kinds['NAME']==row['speciality_of_doctor']]['ID'])
  except:
    pass
  if id==-1:
    id=porydkoviy
    porydkoviy+=1
  return id
df['speciality_of_doctor']=df.apply(doctor_id, axis=1)

In [18]:
#Удаление знаков препинания
def delete_dots(row):
  global operable
  value=row[operable]
  value=list(str(value))
  replaceables=set([',', '.', '?', '/', '-', '_', '(', ')'])
  for i in range(len(value)):
    if value[i] in replaceables:
      value[i]=' '
  value_txt=""
  for i in value:
    value_txt=value_txt+i

  value=' '.join(value_txt.split())
  return value.lower()

operable=""
for column in ['speciality_of_doctor', 'problems', "past_illnesses", "postponed_operations", "medicicals_taken"]:
  operable=column
  df[operable] = df.apply(delete_dots, axis=1)

In [19]:
#Проверка
df.head(5)

,age,sex,speciality_of_doctor,problems,past_illnesses,postponed_operations,medicicals_taken
0,26,1.0,100000,на мажущие выделения из половых путей,факт получения травм ожогов ран проведения медицинских манипуляций инъекций стоматологических хирургических вмешательств и др в течение последних 30 дней отрицает установление имплантата в течении 1 года отрицает перенесенные инфекционные заболевания в т ч вич вирусные гепатиты туберкулез covid 19 отрицает,отрицает,левотироксин 75 мкг сут
1,37,1.0,100001,на дискомфорт при мочеиспускании на учащенное мочеиспускание,факт получения травм ожогов ран проведения медицинских манипуляций инъекций стоматологических хирургических вмешательств и др в течение последних 30 дней отрицает установление имплантата в течении 1 года отрицает перенесенные инфекционные заболевания в т ч вич вирусные гепатиты туберкулез covid 19 отрицает,аппендэктомия 2013 кесарево сечение преэклампсия 2018,монурал канефрон
2,28,0.0,100002,на покраснение и шелушение кожи головы,гэрб жда,,колекальльциферол 4000 ме калия йодид мальтофер 2 т день
3,62,1.0,100003,подписано информированное добровольное согласие на получение первичной медико санитарной помощи прием кардиолога повторно жалобы на фоне стресса повышение ад 145 90 ммрттс с головными болями 2 балла по ваш на эпизоды учащенного сердцебиения без четкой связи с физической нагрузкой перебои в работе сердца,узловой зоб,аппендицит в 15 летнем возрасте удаление кисты бартолиниевой железы в 2007 г в 2022г мастэктоия слева,на пенициллиновый ряд отек квинке сульфаниламиды крапивница на метронидазол чувство дурноты
4,42,1.0,100004,жалобы на болезненность и нагрубание правой молочной железы,факт получения травм ожогов ран проведения медицинских манипуляций инъекций стоматологических хирургических вмешательств и др в течение последних 30 дней отрицает установление имплантата в течении 1 года отрицает перенесенные инфекционные заболевания в т ч вич вирусные гепатиты туберкулез covid 19 отрицает,аппендэктомия в 5 лет импланты молочных желез 2016 лазерная коррекция миопии 2017,не отягощен


In [20]:
df.shape

(12765, 7)

In [21]:
#Удаление пустой строчки в стобце "sex" и привидение его к целочисленному формату
df=df.dropna(subset='sex')
df['sex']=df['sex'].astype(int)

<ipython-input-21-78b5bb218486>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sex']=df['sex'].astype(int)


In [22]:
#Обнуление строк, в которых нет информации о заболеваниях
def nullify_rows(text, texts_to_nullify):
    for substring in texts_to_nullify:
        if substring in text:
            return '0'
    return text

texts_to_nullify = ["факт получения травм ожогов ран проведения медицинских манипуляций", "-", "отриц", "не бы", "не отмеч", "отр", "нет", "не отмечает"]

df['past_illnesses'] = df['past_illnesses'].apply(lambda x: nullify_rows(x, texts_to_nullify))

In [23]:
#Обнуление строк, в которых нет информации о операциях

df["postponed_operations"] = df["postponed_operations"].replace("отриц", 0, regex = True)
df["postponed_operations"] = df["postponed_operations"].replace("отр", 0)
df["postponed_operations"] = df["postponed_operations"].replace("нет", 0)
df["postponed_operations"] = df["postponed_operations"].replace("не бы", 0, regex = True)
df["postponed_operations"] = df["postponed_operations"].replace("-", 0)
df["postponed_operations"] = df["postponed_operations"].replace("nan", 0)
df["postponed_operations"] = df["postponed_operations"].replace("", 0)


In [24]:
#Обнуление строк, в которых нет информации о принимаемых лекарствах
df["medicicals_taken"] = df["medicicals_taken"].replace("не отяг", 0, regex = True)
df["medicicals_taken"] = df["medicicals_taken"].replace("отриц", 0, regex = True)
df["medicicals_taken"] = df["medicicals_taken"].replace("отр", 0)
df["medicicals_taken"] = df["medicicals_taken"].replace("нет", 0)
df["medicicals_taken"] = df["medicicals_taken"].replace("базисной терапии нет", 0, regex = True)
df["medicicals_taken"] = df["medicicals_taken"].replace("-", 0)
df["medicicals_taken"] = df["medicicals_taken"].replace("nan", 0)
df["medicicals_taken"] = df["medicicals_taken"].replace("неизвестен", 0)
df["medicicals_taken"] = df["medicicals_taken"].replace("не принимает", 0, regex = True)


In [25]:
#Вывод названия заболеваний
def extract_noun_and_adjective(text):
    if text == 0:
        return 0
    words = re.findall(r'\b\w+\b', text)
    result = []

    for word in words:
        parsed_word = morph.parse(word)[0]
        if 'NOUN' in parsed_word.tag and 'nomn' in parsed_word.tag:
            result.append(parsed_word.word)


    return ', '.join(result)


df['proc_past_illnesses'] = df['past_illnesses'].apply(extract_noun_and_adjective)

In [26]:
#Проверка
df.head(200)

,age,sex,speciality_of_doctor,problems,past_illnesses,postponed_operations,medicicals_taken,proc_past_illnesses
0,26,1,100000,на мажущие выделения из половых путей,0,0,левотироксин 75 мкг сут,
1,37,1,100001,на дискомфорт при мочеиспускании на учащенное мочеиспускание,0,аппендэктомия 2013 кесарево сечение преэклампсия 2018,монурал канефрон,
2,28,0,100002,на покраснение и шелушение кожи головы,гэрб жда,0,колекальльциферол 4000 ме калия йодид мальтофер 2 т день,
3,62,1,100003,подписано информированное добровольное согласие на получение первичной медико санитарной помощи прием кардиолога повторно жалобы на фоне стресса повышение ад 145 90 ммрттс с головными болями 2 балла по ваш на эпизоды учащенного сердцебиения без четкой связи с физической нагрузкой перебои в работе сердца,узловой зоб,аппендицит в 15 летнем возрасте удаление кисты бартолиниевой железы в 2007 г в 2022г мастэктоия слева,на пенициллиновый ряд отек квинке сульфаниламиды крапивница на метронидазол чувство дурноты,зоб
4,42,1,100004,жалобы на болезненность и нагрубание правой молочной железы,0,аппендэктомия в 5 лет импланты молочных желез 2016 лазерная коррекция миопии 2017,0,
5,35,1,100005,на боли в левом коленном суставе,0,лазерная коррекция зрения,0,
6,36,0,100006,на повышение ад до 150\100 мм рт ст,0,0,0,
7,58,1,100007,жалобы на отрыжку периодическую тяжесть в животе эпигастральной области правом подреберье после еды периодическое вздутие живота дискомфорт при дефекации,0,мастэктомия левой м ж июль 2022 комбинированное лечение,0,
8,40,1,100008,жалобы на частые головные боли ноющие по ваш на 5 6б чаще в затылочной области до тошноты двоение в глазах появление молний лучей и вспышек чаще утром фото и фонофобия присутствует при движении головная боль усиливается жалобы на боли в шейном поясничном отделе позвоночника по ваш на 5 6 иногда онемение левой руки периодически,0,0,0,
9,34,0,100009,на ноющие боли в правом подреберье,0,0,0,


In [27]:
#Проверка размеров
df.shape

(12764, 8)

In [28]:
#Удаление знаков препинания
def delete_dots(row):
  global operable
  value=row[operable]
  value=list(str(value))
  replaceables=set([',', '.', '?', '/', '-', '_', '(', ')', '[', ']', "'"])
  for i in range(len(value)):
    if value[i] in replaceables:
      value[i]=' '
  value_txt=""
  for i in value:
    value_txt=value_txt+i

  value=' '.join(value_txt.split())
  return value.lower()

operable=""
for column in ["proc_past_illnesses"]:
  operable=column
  df[operable] = df.apply(delete_dots, axis=1)

In [29]:
#Замена орз на орви
df['proc_past_illnesses'] = df['proc_past_illnesses'].str.replace('орз', 'орви')

In [30]:
#Подсчет количества повторений
df['proc_past_illnesses'] = df['proc_past_illnesses'].apply(lambda x: word_tokenize(x.lower()))

word_counts = Counter()

for _, row in df.iterrows():
    word_counts.update(row['proc_past_illnesses'])

for word, count in word_counts.items():
    pass

In [31]:
#Удаление знаков препинания
df['proc_past_illnesses']=df['proc_past_illnesses'].astype(str)
operable=""
for column in ["proc_past_illnesses"]:
  operable=column
  df[operable] = df.apply(delete_dots, axis=1)

In [32]:
#Проверка
df.head(3)

,age,sex,speciality_of_doctor,problems,past_illnesses,postponed_operations,medicicals_taken,proc_past_illnesses
0,26,1,100000,на мажущие выделения из половых путей,0,0,левотироксин 75 мкг сут,
1,37,1,100001,на дискомфорт при мочеиспускании на учащенное мочеиспускание,0,аппендэктомия 2013 кесарево сечение преэклампсия 2018,монурал канефрон,
2,28,0,100002,на покраснение и шелушение кожи головы,гэрб жда,0,колекальльциферол 4000 ме калия йодид мальтофер 2 т день,


In [33]:
#Проверка размеров
df.shape

(12764, 8)

In [34]:
#Удаление некоторых значений
word_to_remove_1 = 'болезнь'
word_to_remove_2 = 'ст'
word_to_remove_3 = 'ремиссия'

if word_to_remove_1 in word_counts:
    del word_counts[word_to_remove_1]
if word_to_remove_2 in word_counts:
    del word_counts[word_to_remove_2]
if word_to_remove_3 in word_counts:
    del word_counts[word_to_remove_3]

In [35]:
# Вывод 10 самых релевантных ответов
sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

for word, count in sorted_word_counts[:10]:
    print(f'{word}: {count} раз')

орви: 412 раз
гастрит: 271 раз
тонзиллит: 146 раз
цистит: 77 раз
пиелонефрит: 62 раз
зоб: 60 раз
гипотиреоз: 55 раз
анемия: 43 раз
панкреатит: 37 раз
киста: 36 раз


In [36]:
#Проверка
df.head(3)

,age,sex,speciality_of_doctor,problems,past_illnesses,postponed_operations,medicicals_taken,proc_past_illnesses
0,26,1,100000,на мажущие выделения из половых путей,0,0,левотироксин 75 мкг сут,
1,37,1,100001,на дискомфорт при мочеиспускании на учащенное мочеиспускание,0,аппендэктомия 2013 кесарево сечение преэклампсия 2018,монурал канефрон,
2,28,0,100002,на покраснение и шелушение кожи головы,гэрб жда,0,колекальльциферол 4000 ме калия йодид мальтофер 2 т день,


In [37]:
#Проверка размеров
df.shape

(12764, 8)

In [38]:
#Создание новых столбцов
labels = [word for word, _ in sorted_word_counts]

for i in list(labels)[:10]:
    df.insert(len(list(df)), i, 0)
df.head()

,age,sex,speciality_of_doctor,problems,past_illnesses,postponed_operations,medicicals_taken,proc_past_illnesses,орви,гастрит,тонзиллит,цистит,пиелонефрит,зоб,гипотиреоз,анемия,панкреатит,киста
0,26,1,100000,на мажущие выделения из половых путей,0,0,левотироксин 75 мкг сут,,0,0,0,0,0,0,0,0,0,0
1,37,1,100001,на дискомфорт при мочеиспускании на учащенное мочеиспускание,0,аппендэктомия 2013 кесарево сечение преэклампсия 2018,монурал канефрон,,0,0,0,0,0,0,0,0,0,0
2,28,0,100002,на покраснение и шелушение кожи головы,гэрб жда,0,колекальльциферол 4000 ме калия йодид мальтофер 2 т день,,0,0,0,0,0,0,0,0,0,0
3,62,1,100003,подписано информированное добровольное согласие на получение первичной медико санитарной помощи прием кардиолога повторно жалобы на фоне стресса повышение ад 145 90 ммрттс с головными болями 2 балла по ваш на эпизоды учащенного сердцебиения без четкой связи с физической нагрузкой перебои в работе сердца,узловой зоб,аппендицит в 15 летнем возрасте удаление кисты бартолиниевой железы в 2007 г в 2022г мастэктоия слева,на пенициллиновый ряд отек квинке сульфаниламиды крапивница на метронидазол чувство дурноты,зоб,0,0,0,0,0,0,0,0,0,0
4,42,1,100004,жалобы на болезненность и нагрубание правой молочной железы,0,аппендэктомия в 5 лет импланты молочных желез 2016 лазерная коррекция миопии 2017,0,,0,0,0,0,0,0,0,0,0,0


In [39]:
#Заполнение новых столбцов
cols=labels[:10]
def parser(row):
    dat=row['proc_past_illnesses'].split()
    flag=0
    for i in dat:
        if i in cols:
            row[i]=1
            flag=1
    return row

df=df.apply(parser, axis=1)

In [40]:
#Проверка
df.head(5)

,age,sex,speciality_of_doctor,problems,past_illnesses,postponed_operations,medicicals_taken,proc_past_illnesses,орви,гастрит,тонзиллит,цистит,пиелонефрит,зоб,гипотиреоз,анемия,панкреатит,киста
0,26,1,100000,на мажущие выделения из половых путей,0,0,левотироксин 75 мкг сут,,0,0,0,0,0,0,0,0,0,0
1,37,1,100001,на дискомфорт при мочеиспускании на учащенное мочеиспускание,0,аппендэктомия 2013 кесарево сечение преэклампсия 2018,монурал канефрон,,0,0,0,0,0,0,0,0,0,0
2,28,0,100002,на покраснение и шелушение кожи головы,гэрб жда,0,колекальльциферол 4000 ме калия йодид мальтофер 2 т день,,0,0,0,0,0,0,0,0,0,0
3,62,1,100003,подписано информированное добровольное согласие на получение первичной медико санитарной помощи прием кардиолога повторно жалобы на фоне стресса повышение ад 145 90 ммрттс с головными болями 2 балла по ваш на эпизоды учащенного сердцебиения без четкой связи с физической нагрузкой перебои в работе сердца,узловой зоб,аппендицит в 15 летнем возрасте удаление кисты бартолиниевой железы в 2007 г в 2022г мастэктоия слева,на пенициллиновый ряд отек квинке сульфаниламиды крапивница на метронидазол чувство дурноты,зоб,0,0,0,0,0,1,0,0,0,0
4,42,1,100004,жалобы на болезненность и нагрубание правой молочной железы,0,аппендэктомия в 5 лет импланты молочных желез 2016 лазерная коррекция миопии 2017,0,,0,0,0,0,0,0,0,0,0,0


In [41]:
#Проверка размера
df.shape

(12764, 18)

In [42]:
#Выгрузка окончательного результата
df.to_csv('res.csv', index=False)